In [4]:
#music genre classifier

#extract inputs and targets (labels & MFCCs)
#store in JSON file
#use to train neural network


import os
import librosa
import math
import json

In [8]:
#download GENRE dataset
#http://marsyas.info/downloads/datasets.html
DATASET_PATH = "E:/DATASETS/soundAnalytics/genre_DS_reduced" #sample of 1 audio per genre
JSON_PATH = "data.json"

#define parameters/constants
SAMPLE_RATE = 22050
DURATION = 30 #seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

In [10]:
#preprocess

#function that saves MFCCs


def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5): 
    #num_segments ---- chop up each track into segments for more input data
    
    #build dictionary to store data
    data = {
        "mapping":[], #map genres to numbers
        "mfcc": [],   #training data
        "labels": []  #genre label - in numerical form
    }
    
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    
    #since some segments may be longer
    expected_num_mfcc_vectors_per_second = math.ceil(num_samples_per_segment/hop_length) #1.3 -> 2
    
    #loop through all genres and analyse individually --- i & enumarate == at each iteration we are in a genre
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        
        #ensure we aren't at the root directory - i.e. dataset path -- but at the genre subfolder
        if dirpath is not dataset_path:
            
            #save semantic label
            dirpath_components = dirpath.split("/")  #genre/blues ===gives a list===> ["genre", "blues"]
            semantic_label = dirpath_components[-1]  #give us the last index value .. ie. "blues"
            data["mapping"].append(semantic_label)
            print("\n Processing {}".format(semantic_label)) #at genre level
            
            #process files for a specific genre
            for f in filenames:
                
                #get file path
                file_path = os.path.join(dirpath, f) #pass the directory path and file name
                #load audio file
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
                
                
                #divide signal into segments
                #process segments ... extract mfcc ... store data
                for s in range(num_segments):      #for the current segment we're in (i.e. 's')
                    start_sample = num_samples_per_segment * s  #initial s=0
                    finish_sample = start_sample + num_samples_per_segment
                    
                    
                    #get mfcc
                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                               sr= sr,
                                               n_fft = n_fft,
                                               n_mfcc = n_mfcc,         #per the current segment: 
                                               hop_length = hop_length) #we want to analyze a slice of the signal 
                    
                    mfcc = mfcc.T
                
                    #store mfcc segment ONLY if it has expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_second:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1) #to come out of the genre label into the dataset home
                        print("{}, segment:{} \n".format(file_path, s++1) ) 
                        
        with open(json_path, "w") as fp:
            json.dump(data, fp, indent=4) #write dictionary(data) to file
                   

In [11]:
if __name__ == "__main__":
    save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)


 Processing genre_DS_reduced\blues
E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:1 

E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:2 

E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:3 

E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:4 

E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:5 

E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:6 

E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:7 

E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:8 

E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:9 

E:/DATASETS/soundAnalytics/genre_DS_reduced\blues\blues.00000.wav, segment:10 


 Processing genre_DS_reduced\classical
E:/DATASETS/soundAnalytics/genre_DS_reduced\classical\classical.00000.wav, segment:1 

E:/DATASETS/soundAnalytics/genre_DS_reduced\cl